# Visualization API Example

This notebook shows how how to use the `visualization` library for viewing images stored on disk.

In [1]:
import sys
if '../' not in sys.path:
    sys.path.append("../")

import matplotlib.pyplot as plt
    
import storage
import visualization

# TODO: Change this path to the directory containing predictions.
PREDICTIONS_DATASET_PATH = "/tmp/ipis"

%matplotlib inline

In [2]:
# Open an on-disk image dataset.
dataset = storage.DiskDataset(PREDICTIONS_DATASET_PATH)

# pd.DataFrame listing all (location_id, image_source_id, metadata).
dataset.metadata().sort_values(["location_id", "source_id"]).sample(5)

,location_id,metadata,source_id
283,codmine00400,"{u'dates': [u'20161015', u'20161116', u'201612...",landsat8
21,codmine00071,"{u'dates': [u'20161015', u'20161116', u'201612...",landsat8
257,codmine00331,"{u'dates': [u'20161015', u'20161116', u'201612...",landsat8
201,codmine00399,"{u'dates': [u'20161015', u'20161116', u'201612...",landsat8
88,codmine01931,"{u'dates': [u'20161015', u'20161116', u'201612...",landsat8


## Visualizing a Single Image

The following blocks visualize a single satellite image and mask combination. For example, one might use 'landsat8' for `IMAGERY_SOURCE_ID` and 'mask' for `MASK_SOURCE_ID`. 

Both strings should in `dataset.metadata().source_id` from the dataset. The `LOCATION_ID` denotes which location to draw the image from, and must be a in `dataset.metadata().location_id`.

In [ ]:
# Define which location you want.

# TODO: Name of location to visualize.
LOCATION_ID = "codmine00126"

# TODO: Name of source to use for visualizing geography.
IMAGERY_SOURCE_ID = "landsat8"

# TODO: Name of source to use for mining masks, predictions, etc.
MASK_SOURCE_ID = "mask"

In [ ]:
# Loads image from disk.

image = dataset.load_image(LOCATION_ID, IMAGERY_SOURCE_ID)
mask = dataset.load_image(LOCATION_ID, MASK_SOURCE_ID)
meta = dataset.image_metadata(LOCATION_ID, IMAGERY_SOURCE_ID)["metadata"]

# TODO: If the image array has more than rank 3, select which slice to use 
# now. For example, Landsat 8 images have a date dimension.
image = image[..., 2]
mask = mask

assert len(image.shape) == 3, "image should have shape [longitude, latitude, color band]."
assert len(mask.shape) == 2, "mask should have shape [longitude, latitude]."

# Derive bounding box around image. 
bounding_box = visualization.BoundingBox.from_metadata(meta)

In [ ]:
# Print out where the image is.

longitude, latitude = bounding_box.center
print("longitude = %f | latitude = %f" % (longitude, latitude))
visualization.show_google_maps_link(longitude, latitude)

In [ ]:
# Visualize image.

fig, axs = plt.subplots(1, 2, figsize=(16, 16))

image_rgb = (
    visualization.landsat8_color_combination(image, "Natural Color") 
    if IMAGERY_SOURCE_ID 
    else image[..., 0:3])

plt.sca(axs[0])
visualization.show_image(
    image_rgb,
    visualization.BoundingBox.from_metadata(meta), 
    title="Landsat 8")

plt.sca(axs[1])
visualization.show_image(
    mask,
    visualization.BoundingBox.from_metadata(meta), 
    title="Mines")

In [ ]:
# Visualize color bands.
    
plt.figure(figsize=(16, 10))
visualization.show_color_bands(image, ncols=4, titles=meta['bands'])

In [ ]:
# View recommended color combinations for Landsat 8 imagery.

if IMAGERY_SOURCE_ID is "landsat8":
    plt.figure(figsize=(16, 12))
    visualization.show_landsat8_color_combinations(image, ncols=4)
else:
    print("This visualization is only available when IMAGERY_SOURCE_ID == 'landsat8'.")

In [ ]:
# Build map. Overlay image on top.

folium_map = visualization.create_folium_map(
    bounding_box,
    tiles='ArcGIS')
visualization.overlay_image_on_map(
    folium_map, 
    image_rgb,
    bounding_box,
    opacity=0.75)
visualization.overlay_image_on_map(
    folium_map, 
    mask, 
    bounding_box,
    opacity=0.2)

folium_map

## Visualizing Many Images

Use the following to overlay multiple images at once. All images from each source will be overlaid over a base map, in order.

In [7]:
# Define which images to overlay.

# TODO: Choose which Locations and sources to load images for.
LOCATION_IDS = dataset.metadata().location_id.unique()[0:100]
SOURCE_IDS = ['landsat8', 'mask']

# TODO: Define an image preprocessing function. If the raw image loaded
# from 'dataset' isn't already grayscale, RGB, or RGBA, make it so here.
def image_preprocessing_fn(image, metadata, location_id, source_id):
    if source_id == 'landsat8':
        return visualization.landsat8_color_combination(image[..., 2], "Natural Color")
    if source_id == 'mask':
        return image
    else:
        raise ValueError("Does source_id = %s need preprocessing?" % source_id)

In [11]:
# Overlay images on map.

# Ensure that folium won't die on us.
num_images = len(LOCATION_IDS) * len(SOURCE_IDS)
if num_images > 80:
    print("""
**WARNINGS!!** You've asked for %d images. If you are using Chrome, you cannot
display > 80 images. Firefox does not have this issue. Ensure jupyter notebook
was launched with --NotebookApp.iopub_data_rate_limit=10000000000 to ensure the
following will display.

See https://github.com/python-visualization/folium/issues/768."""
        % (num_images,))
    
    
# Create map to add images to.
folium_map = visualization.create_folium_map(tiles='ArcGIS')

# Overlay images on map.
for source_id in SOURCE_IDS:
    visualization.overlay_images_on_map(
        dataset, 
        folium_map, 
        LOCATION_IDS, 
        source_id, 
        image_preprocessing_fn=image_preprocessing_fn, 
        opacity=0.5)

folium_map


**WARNINGS!!** You've asked for 200 images. If you are using Chrome, you cannot
display > 80 images. Firefox does not have this issue. Ensure jupyter notebook
was launched with --NotebookApp.iopub_data_rate_limit=10000000000 to ensure the
following will display.
        
